In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df=pd.read_csv('covid_toy.csv')

In [4]:
df

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No
...,...,...,...,...,...,...
95,12,Female,104.0,Mild,Bangalore,No
96,51,Female,101.0,Strong,Kolkata,Yes
97,20,Female,101.0,Mild,Bangalore,No
98,5,Female,98.0,Strong,Mumbai,No


In [5]:
df.columns

Index(['age', 'gender', 'fever', 'cough', 'city', 'has_covid'], dtype='object')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        100 non-null    int64  
 1   gender     100 non-null    object 
 2   fever      90 non-null     float64
 3   cough      100 non-null    object 
 4   city       100 non-null    object 
 5   has_covid  100 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 4.8+ KB


In [7]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [8]:
df.describe()

,age,fever
count,100.000000,90.000000
mean,44.220000,100.844444
std,24.878931,2.054926
min,5.000000,98.000000
25%,20.000000,99.000000
50%,45.000000,101.000000
75%,66.500000,102.750000
max,84.000000,104.000000


In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

In [10]:
X_train.shape

(80, 5)

In [11]:
X_test.shape

(20, 5)

In [12]:
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

**AAM ZINDEGI**

In [13]:
#Simple Imputer is used to replace missing values in a dataset, using a variety of input strategies  in fever col
si=SimpleImputer()
X_train_fever=si.fit_transform(X_train[['fever']])
#Also test the data
X_test_fever=si.fit_transform(X_test[['fever']])
X_train_fever

array([[ 98.        ],
       [100.90277778],
       [100.        ],
       [ 98.        ],
       [102.        ],
       [102.        ],
       [101.        ],
       [ 98.        ],
       [104.        ],
       [101.        ],
       [100.90277778],
       [100.90277778],
       [ 98.        ],
       [101.        ],
       [100.        ],
       [100.90277778],
       [104.        ],
       [ 99.        ],
       [ 99.        ],
       [104.        ],
       [104.        ],
       [100.        ],
       [ 99.        ],
       [101.        ],
       [104.        ],
       [ 98.        ],
       [100.        ],
       [101.        ],
       [103.        ],
       [102.        ],
       [101.        ],
       [101.        ],
       [104.        ],
       [102.        ],
       [102.        ],
       [ 99.        ],
       [100.        ],
       [100.        ],
       [100.        ],
       [103.        ],
       [104.        ],
       [100.        ],
       [100.90277778],
       [ 98

In [14]:
#Onehotencoder used for cough categorical data(used for ordinal data=having heirarchy)
oe=OrdinalEncoder()
X_train_cough=oe.fit_transform(X_train[['cough']])
X_test_cough=oe.fit_transform(X_train[['cough']])
X_train_cough.shape

(80, 1)

In [15]:
#onehotencoding for gender and city
oh=OneHotEncoder(drop='first',sparse=False)
X_train_gender_city=oh.fit_transform(X_train[['gender','city']])
X_test_gender_city=oh.fit_transform(X_train[['gender','city']])
X_train_gender_city.shape

(80, 4)

In [16]:
#Extracting the a('gender','cough','city','fver']).values
X_train_age=X_train.drop(columns=['gender','fever',	'cough','city'])
X_test_age=X_train.drop(columns=['gender','fever',	'cough','city'])


In [17]:
#Extracting the age and testing the age
X_train_transformed=np.concatenate((X_train_age,X_train_gender_city,X_train_fever,X_train_cough),axis=1)
X_test_transformed=np.concatenate((X_test_age,X_test_gender_city,X_test_fever,X_train_cough),axis=1)                                 

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 80 and the array at index 2 has size 20

In [18]:
X_train_transformed.shape

(80, 7)

**MENTOS ZINDAGI**

In [19]:
from sklearn.compose import ColumnTransformer

In [20]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [21]:
X_train_transformed=transformer.fit_transform(X_train)

In [22]:
X_train_transformed.shape

(80, 7)

In [23]:
transformer.transform(X_test).shape

(20, 7)